In [1]:
!pip install kubernetes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.3 MB/s eta 0:00:0000:01
  Using cached websocket_client-1.3.3-py3-none-any.whl (54 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 59.3 MB/s eta 0:00:00


In [2]:
from kubernetes import client, config

In [3]:
# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

In [6]:
v1 = client.CoreV1Api()
print("Listing pods with their IPs:")
ret = v1.list_pod_for_all_namespaces(watch=False)
for i in ret.items:
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

Listing pods with their IPs:
10.244.2.8	auth	dex-5ddf47d88d-mgfhw
10.244.1.10	cert-manager	cert-manager-7b8c77d4bd-7lb4c
10.244.3.8	cert-manager	cert-manager-cainjector-7c744f57b5-hplmn
10.244.4.12	cert-manager	cert-manager-webhook-fcd445bc4-vgd8j
10.244.4.72	default	sts-ubuntu-0
10.244.3.35	default	sts-ubuntu-1
10.244.1.29	default	sts-ubuntu-2
10.244.2.33	default	sts-ubuntu-3
10.244.4.2	ingress-nginx	ingress-nginx-admission-create-b7zqn
10.244.4.3	ingress-nginx	ingress-nginx-admission-patch-nwgfj
10.244.0.5	ingress-nginx	ingress-nginx-controller-9549c9fb9-5qffh
10.244.4.15	istio-system	authservice-0
10.244.1.17	istio-system	cluster-local-gateway-64f58f66cb-pm6fw
10.244.3.16	istio-system	istio-ingressgateway-8577c57fb6-c5cvm
10.244.3.9	istio-system	istiod-6c86784695-p2x6g
10.244.2.9	knative-eventing	eventing-controller-79895f9c56-c94d2
10.244.3.10	knative-eventing	eventing-webhook-78f897666-4g6s5
10.244.1.11	knative-eventing	imc-controller-688df5bdb4-kmnfh
10.244.4.13	knative-eventing	

In [9]:
from kubernetes.client import ApiException

api_instance = client.CoreV1Api()
namespace = "tf-serve-test"

serve_service = {
    "apiVersion": "v1",
    "kind": "Service",
    "metadata": {
        "labels": {
            "app": "sentiment-analysis"
        },
        "name": "sentiment-analysis-service",
        "namespace": namespace
    },
    "spec": {
        "ports": [
            {
                "name": "grpc-tf-serving",
                "port": 8500,
                "targetPort": 8500
            },
            {
                "name": "http-tf-serving",
                "port": 8501,
                "targetPort": 8501
            }
        ],
        "selector": {
            "app": "sentiment-analysis"
        },
        "type": "ClusterIP"
    }
}
try:
    api_response = api_instance.create_namespaced_service(namespace, serve_service, pretty='true')
    print(api_response)
except ApiException as e:
    print("Exception when calling CoreV1Api->create_namespaced_endpoints: %s\n" % e)

Exception when calling CoreV1Api->create_namespaced_endpoints: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'X-Kubernetes-Pf-Flowschema-Uid': '24ba791f-127e-4a60-b4f7-ec4c6d1e4e0a', 'X-Kubernetes-Pf-Prioritylevel-Uid': 'daf5a69d-cbb6-4142-99fd-2ba57018f70b', 'Date': 'Wed, 10 Aug 2022 04:15:44 GMT', 'Content-Length': '293'})
HTTP response body: {
  "kind": "Status",
  "apiVersion": "v1",
  "metadata": {
    
  },
  "status": "Failure",
  "message": "services \"sentiment-analysis-service\" already exists",
  "reason": "AlreadyExists",
  "details": {
    "name": "sentiment-analysis-service",
    "kind": "services"
  },
  "code": 409
}




In [11]:
apps_v1 = client.AppsV1Api()
serve_deployment = {
    "apiVersion": "apps/v1",
    "kind": "Deployment",
    "metadata": {
        "labels": {
            "app": "sentiment-analysis"
        },
        "name": "sentiment-analysis-v1",
        "namespace": namespace
    },
    "spec": {
        "selector": {
            "matchLabels": {
                "app": "sentiment-analysis"
            }
        },
        "template": {
            "metadata": {
                "annotations": {
                    "sidecar.istio.io/inject": "true"
                },
                "labels": {
                    "app": "sentiment-analysis",
                    "version": "v1"
                }
            },
            "spec": {
                "containers": [
                    {
                        "args": [
                            "--enable_batching",
                            "--port=8500",
                            "--rest_api_port=8501",
                            "--model_name=sentiment-analysis",
                            "--model_base_path=s3://datasets/imdb_sentiment_analysis"
                        ],
                        "command": [
                            "/usr/bin/tensorflow_model_server"
                        ],
                        "image": "tensorflow/serving",
                        "imagePullPolicy": "IfNotPresent",
                        "livenessProbe": {
                            "initialDelaySeconds": 30,
                            "periodSeconds": 30,
                            "tcpSocket": {
                                "port": 9000
                            }
                        },
                        "name": "sentiment-analysis",
                        "ports": [
                            {
                                "containerPort": 9000
                            },
                            {
                                "containerPort": 8500
                            }
                        ],
                        "env":[
                            {
                                "name":"AWS_ACCESS_KEY_ID",
                                "value": "kubeflow"
                            },
                            {
                                "name":"AWS_SECRET_ACCESS_KEY",
                                "value": "kubeflow123"
                            },
                            {
                                "name":"AWS_REGION",
                                "value": "us-east-1"
                            },
                            {
                                "name":"S3_ENDPOINT",
                                "value": "minio.ns-1.svc.cluster.local"
                            },
                            {
                                "name":"S3_USE_HTTPS",
                                "value": "0"
                            },
                            {
                                "name":"S3_VERIFY_SSL",
                                "value": "0"
                            },
                        ],
                        "resources": {
                            "limits": {
                                "cpu": "4",
                                "memory": "4Gi"
                            },
                            "requests": {
                                "cpu": "1",
                                "memory": "1Gi"
                            }
                        }
                    }
                ]
            }
        }
    }
}
try:
    api_response = apps_v1.create_namespaced_deployment(namespace, serve_deployment, pretty='true')
    print(api_response)
except ApiException as e:
    print("Exception when calling CoreV1Api->create_namespaced_endpoints: %s\n" % e)

{'api_version': 'apps/v1',
 'kind': 'Deployment',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2022, 8, 10, 4, 25, 51, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 1,
              'labels': {'app': 'sentiment-analysis'},
              'managed_fields': [{'api_version': 'apps/v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:metadata': {'f:labels': {'.': {},
                                                                            'f:app': {}}},
                                                'f:spec': {'f:progressDeadlineSeconds': {},
                                                           'f:replicas': {},
                                                           'f:revi

In [15]:
serve_destination_rule = {
    "apiVersion": "networking.istio.io/v1alpha3",
    "kind": "DestinationRule",
    "metadata": {
        "name": "sentiment-analysis-service",
        "namespace": namespace
    },
    "spec": {
        "host": "sentiment-analysis-service",
        "subsets": [
            {
                "labels": {
                    "version": "v1"
                },
                "name": "v1"
            }
        ]
    }
}
cust_api = client.CustomObjectsApi()

try:
    api_response = cust_api.create_namespaced_custom_object(
        namespace=namespace,
        group="networking.istio.io",
        version="v1alpha3",
        plural="destinationrules",
        body=serve_destination_rule,
        pretty='true')
    print(api_response)
except ApiException as e:
    print("Exception when calling CoreV1Api->create_namespaced_endpoints: %s\n" % e)


{'apiVersion': 'networking.istio.io/v1alpha3', 'kind': 'DestinationRule', 'metadata': {'creationTimestamp': '2022-08-10T04:29:48Z', 'generation': 1, 'managedFields': [{'apiVersion': 'networking.istio.io/v1alpha3', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:spec': {'.': {}, 'f:host': {}, 'f:subsets': {}}}, 'manager': 'OpenAPI-Generator', 'operation': 'Update', 'time': '2022-08-10T04:29:48Z'}], 'name': 'sentiment-analysis-service', 'namespace': 'tf-serve-test', 'resourceVersion': '4333006', 'uid': '2848226d-e66e-471c-aeed-b8aa65369316'}, 'spec': {'host': 'sentiment-analysis-service', 'subsets': [{'labels': {'version': 'v1'}, 'name': 'v1'}]}}


In [16]:
serve_virtual_service = {
    "apiVersion": "networking.istio.io/v1alpha3",
    "kind": "VirtualService",
    "metadata": {
        "name": "sentiment-analysis-service",
        "namespace": namespace
    },
    "spec": {
        "gateways": [
            "kubeflow-gateway"
        ],
        "hosts": [
            "*"
        ],
        "http": [
            {
                "match": [
                    {
                        "method": {
                            "exact": "POST"
                        },
                        "uri": {
                            "prefix": "/tfserving/models/sentiment-analysis"
                        }
                    }
                ],
                "rewrite": {
                    "uri": "/v1/models/sentiment-analysis:predict"
                },
                "route": [
                    {
                        "destination": {
                            "host": "sentiment-analysis-service",
                            "port": {
                                "number": 8500
                            },
                            "subset": "v1"
                        },
                        "weight": 100
                    }
                ]
            }
        ]
    }
}

try:
    api_response = cust_api.create_namespaced_custom_object(
        namespace=namespace,
        group="networking.istio.io",
        version="v1alpha3",
        plural="virtualservices",
        body=serve_virtual_service,
        pretty='true')
    print(api_response)
except ApiException as e:
    print("Exception when calling CoreV1Api->create_namespaced_endpoints: %s\n" % e)



{'apiVersion': 'networking.istio.io/v1alpha3', 'kind': 'VirtualService', 'metadata': {'creationTimestamp': '2022-08-10T04:33:08Z', 'generation': 1, 'managedFields': [{'apiVersion': 'networking.istio.io/v1alpha3', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:spec': {'.': {}, 'f:gateways': {}, 'f:hosts': {}, 'f:http': {}}}, 'manager': 'OpenAPI-Generator', 'operation': 'Update', 'time': '2022-08-10T04:33:08Z'}], 'name': 'sentiment-analysis-service', 'namespace': 'tf-serve-test', 'resourceVersion': '4337929', 'uid': '183a4056-5f78-4610-b9f5-100bf7b28792'}, 'spec': {'gateways': ['kubeflow-gateway'], 'hosts': ['*'], 'http': [{'match': [{'method': {'exact': 'POST'}, 'uri': {'prefix': '/tfserving/models/sentiment-analysis'}}], 'rewrite': {'uri': '/v1/models/sentiment-analysis:predict'}, 'route': [{'destination': {'host': 'sentiment-analysis-service', 'port': {'number': 8500}, 'subset': 'v1'}, 'weight': 100}]}]}}
